In [11]:
# Khai báo các thư viện cần thiết
import os
import random
import pickle

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [12]:
# Khai báo tên meta model
while True:
    model_name = input("Enter meta model name: ")
    if model_name.strip() != "":
        break
    else:
        print("Model name cannot be empty. Please enter a valid name.")
os.makedirs(model_name, exist_ok=True)

In [13]:
# Lớp TrainingData 
# Lớp này dùng để lưu trữ dữ liệu huấn luyện cho mô hình

class TrainingData:
    def __init__(self):
        """
        Khởi tạo lớp TrainingData với dữ liệu và nhãn.
        
        :param data: Dữ liệu huấn luyện (danh sách các mẫu).
        :param labels: Nhãn tương ứng với dữ liệu (danh sách các nhãn).
        """
        self.data = []
    
    def add(self, sample):
        """
        Thêm một mẫu vào dữ liệu huấn luyện.
        
        :param sample: Mẫu dữ liệu cần thêm.
        """
        self.data.append(sample)

    def sample(self, batch_size):
        """
        Lấy một mẫu ngẫu nhiên từ dữ liệu huấn luyện với kích thước batch_size.
        
        :param batch_size: Kích thước của mẫu cần lấy.
        :return: Một danh sách chứa các mẫu ngẫu nhiên.
        """
        return random.sample(self.data, batch_size) if len(self.data) >= batch_size else self.data
    
    def __len__(self):
        """
        Trả về số lượng mẫu trong dữ liệu huấn luyện.
        
        :return: Số lượng mẫu trong dữ liệu.
        """
        return len(self.data)

In [14]:
# Nạp dữ liệu huấn luyện vào training_data
training_data = TrainingData()

label = 0  # Biến nhãn khởi tạo bằng 0
models_info = []  # Danh sách chứa thông tin mô hình

for model_folder in os.listdir():
    if os.path.exists(model_folder + "/model.pth") and os.path.exists(model_folder + "/model_info.pkl") and os.path.exists(model_folder + "/dataset"):
        with open(model_folder + "/model_info.pkl", 'rb') as f:
            model_info = pickle.load(f) 
            models_info.append(model_info)
        for data in os.listdir(model_folder + "/dataset"):
            data_path = os.path.join(model_folder + "/dataset", data)
            with open(data_path, 'rb') as f:
                data = pickle.load(f)
                data['label'] = label
                training_data.add(data)
        label += 1  # Tăng nhãn cho thư mục tiếp theo
    else:
        continue
        
print(f"Loaded {len(training_data)} samples from {len(models_info)} classes.")

# Lưu models_info vào file
with open(model_name + "/models_info.pkl", 'wb') as f:
    pickle.dump(models_info, f)
print(f"Saved models_info to {model_name}/models_info.pkl")



Loaded 11171 samples from 3 classes.
Saved models_info to meta_model01/models_info.pkl


In [15]:
# Mạng nơron

class MetaModelNet(nn.Module):
    """
    Mạng nơron cho mô hình MetaModelNet.
    """
    def __init__(self, local_size, num_classes):
        super(MetaModelNet, self).__init__()
    
        # Tầng CNN cho local_obs
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, padding=1)

        # Tầng FC cho position
        self.fc_position = nn.Linear(2, 32)

        # Tầng FC cuối cùng
        self.fc1 = nn.Linear(256 * local_size * local_size + 32, 512)
        self.fc2 = nn.Linear(512, 256)
        
        # Dropout trước tầng FC cuối cùng
        self.dropout = nn.Dropout(0.5)
        # Tầng đầu ra
        self.fc3 = nn.Linear(256, num_classes)

    def forward(self, local_obs, position):
        """
        Hàm truyền dữ liệu qua mạng nơron.
        
        :param local_obs: Dữ liệu đầu vào (local observations).
        :param position: Vị trí của dữ liệu đầu vào.
        :return: Kết quả đầu ra của mạng nơron.
        """
        # Xử lý local_obs qua các tầng CNN
        x_local = F.relu(self.conv1(local_obs))
        x_local = F.relu(self.conv2(x_local))
        x_local = F.relu(self.conv3(x_local))
        x_local = F.relu(self.conv4(x_local))

        # Chuyển đổi kích thước tensor
        x_local = x_local.view(x_local.size(0), -1)  # Chuyển đổi thành vector 1 chiều

        # Xử lý position qua tầng FC
        x_position = F.relu(self.fc_position(position))

        # Kết hợp dữ liệu từ local_obs và position
        x = torch.cat((x_local, x_position), dim=1)

        # Truyền qua các tầng FC
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x        

In [16]:
# Lưu, tải trọng số của mô hình
def save_model(model, filename):
    """
    Lưu trọng số của mô hình vào file.
    
    :param model: Mô hình cần lưu.
    :param filename: Tên file để lưu trọng số.
    """
    torch.save(model, filename)
    print(f"Model saved to {filename}")

def load_model(filename):
    """
    Tải trọng số của mô hình từ file.
    
    :param model: Mô hình cần tải trọng số.
    :param filename: Tên file chứa trọng số.
    """
    if os.path.exists(filename):
        model = torch.load(filename, map_location=device, weights_only=False)
        print(f"Model loaded from {filename}")
        return model
    else:
        print(f"File {filename} does not exist.")
        return MetaModelNet(local_size, num_classes).to(device)

In [17]:
# Khởi tạo các siêu tham số

local_size = 7  # Kích thước của local_obs
num_classes = len(models_info)  # Số lớp (số nhãn)

max_episodes = 50000  # Số lượng episode tối đa

batch_size = 32  # Kích thước batch

learning_rate = 0.001  # Tốc độ học
weight_decay = 0.0001  # Tham số weight decay

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Kiểm tra xem có GPU hay không
print(f"Using device: {device}")

# Khởi tạo mô hình, hàm mất mát và bộ tối ưu hóa
model = load_model(model_name + "/meta_model.pth").to(device)  # Tải mô hình từ file

optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
print("Initialization complete!")

# Lưu thông tin các tham số vào file
with open(model_name + "/meta_model.info", 'w') as f:
    f.write(f"Hyperparameters:\n")
    f.write(f"Max episodes: {max_episodes}\n")
    f.write(f"Batch size: {batch_size}\n")
    f.write(f"Number of samples: {len(training_data)}\n")
    f.write(f"Learning rate: {learning_rate}\n")
    f.write(f"Weight decay: {weight_decay}\n")
    f.write(f"Device: {device}\n")

    f.write(f"\nModel configuration:\n")
    f.write(f"Local size: {local_size}\n")
    f.write(f"Number of classes: {num_classes}\n")

print("Model configuration saved to " + model_name + "/meta_model.info")

Using device: cuda
Model loaded from meta_model01/meta_model.pth
Initialization complete!
Model configuration saved to meta_model01/meta_model.info


In [ ]:
# Vòng lặp huấn luyện
for episode in range(max_episodes):
    #1. Lấy một batch dữ liệu ngẫu nhiên từ training_data
    batch = training_data.sample(batch_size)

    #2. Tách dữ liệu thành các phần riêng biệt
    local_obs = torch.stack([torch.tensor(sample['local_obs'], dtype=torch.float32) for sample in batch])
    agent_start_positions = torch.stack([torch.tensor(sample['agent_start_position'], dtype=torch.float32) for sample in batch])
    agent_end_positions = torch.stack([torch.tensor(sample['agent_end_position'], dtype=torch.float32) for sample in batch])
    labels = torch.tensor([sample['label'] for sample in batch], dtype=torch.long)

    #3. Chuyển dữ liệu sang GPU nếu có
    local_obs = local_obs.to(device)
    agent_start_positions = agent_start_positions.to(device)
    agent_end_positions = agent_end_positions.to(device)
    labels = labels.to(device)

    #4. Tính giá trị target dựa vào agent_start_positions và agent_end_positions
    target_positions = agent_end_positions - agent_start_positions  
    target = torch.sum(target_positions, dim=1)  # Tính tổng các vị trí

    #5. Đưa dữ liệu vào mô hình và tính toán dự đoán
    local_obs = local_obs.unsqueeze(1)  # Thêm chiều cho local_obs
    predictions = model(local_obs, agent_start_positions)
    prediction = predictions[torch.arange(batch_size), labels]

    #6. Tính toán hàm mất mát
    loss = F.mse_loss(prediction, target)
    
    #7. Tối ưu hóa mô hình
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    #8. In thông tin về quá trình huấn luyện
    print(f"Episode {episode + 1}/{max_episodes}, Loss: {loss.item()}")


Episode 1/50000, Loss: 0.6074149012565613
Episode 2/50000, Loss: 0.6103271245956421
Episode 3/50000, Loss: 0.6874008774757385
Episode 4/50000, Loss: 0.3559432327747345
Episode 5/50000, Loss: 0.5341411232948303
Episode 6/50000, Loss: 0.3869292736053467
Episode 7/50000, Loss: 0.7781364917755127
Episode 8/50000, Loss: 0.5937939882278442
Episode 9/50000, Loss: 1.3202606439590454
Episode 10/50000, Loss: 0.5958864688873291
Episode 11/50000, Loss: 0.7391487956047058
Episode 12/50000, Loss: 0.9898227453231812
Episode 13/50000, Loss: 1.0739858150482178
Episode 14/50000, Loss: 0.9449949264526367
Episode 15/50000, Loss: 1.2361308336257935
Episode 16/50000, Loss: 0.9383218288421631
Episode 17/50000, Loss: 0.5590945482254028
Episode 18/50000, Loss: 0.44759517908096313
Episode 19/50000, Loss: 0.8675855398178101
Episode 20/50000, Loss: 0.8470847606658936
Episode 21/50000, Loss: 0.6895284652709961
Episode 22/50000, Loss: 0.5964802503585815
Episode 23/50000, Loss: 0.808462381362915
Episode 24/50000, Lo

KeyboardInterrupt: 

In [19]:
# Lưu trọng số của mô hình
save_model(model, model_name + '/meta_model.pth')  

Model saved to meta_model01/meta_model.pth
